IMPORT LIBRARIES

In [112]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import os

In [113]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
}

keyword = 'shoes'
imageList, linkList, nameList, conditionList, priceList, colorsList, sizesList, locationList = [], [], [], [], [], [], [], []

SCRAPING

In [115]:
for page in range(1, 4):
    URL = f'https://www.ebay.com/sch/i.html?_nkw={keyword}&_pgn={page}'
    res = requests.get(URL, headers=headers)
    pageSource = res.text

    content = BeautifulSoup(pageSource, 'html.parser')
    items = content.findAll('div', 's-item__image-section')
    items.pop(0)


    for item in items:
        image = item.find('div', 's-item__image-wrapper image-treatment').find('img')['src']
        link = item.find('a')['href']
        
        res = requests.get(link, headers=headers)
        content = BeautifulSoup(res.text, 'html.parser')
        product = content.find('div', id='LeftSummaryPanel')

        if product:
            name = product.find('span', 'ux-textspans ux-textspans--BOLD').text
            condition = product.find('div', 'x-item-condition-text').find('span', 'ux-textspans').text
            price = product.find('div', 'x-price-primary').find('span', 'ux-textspans').text
            colors = ''
            sizes = ''

            location = product.find('div', 'ux-labels-values__values col-9').find('span', 'ux-textspans ux-textspans--SECONDARY')
            if location: location = location.text.replace('Located in: ', '')

            for i in range(2):
                option = product.find('select', id=f'x-msku__select-box-100{i}')
                if option:
                    option = product.find('select', id=f'x-msku__select-box-100{i}')['aria-label']
                    if 'Color' in option:
                        option = product.find('select', id=f'x-msku__select-box-100{i}').findAll('option')
                        color = [ops.text.replace('\xa0 (Out Of Stock)', '') for ops in option]
                        color.pop(0)
                        colors = ', '.join(color)

                    if 'Size' in option:
                        option = product.find('select', id=f'x-msku__select-box-100{i}').findAll('option')
                        size = [ops.text.replace('\xa0 (Out Of Stock)', '') for ops in option]
                        size.pop(0)
                        sizes = ', '.join(size)

            imageList.append(image)
            linkList.append(link)
            nameList.append(name)
            conditionList.append(condition)
            priceList.append(price)
            colorsList.append(colors)
            sizesList.append(sizes)
            locationList.append(location)
            
        time.sleep(1)

DATAFRAME

In [ ]:
data = {
    'Product Name': nameList,
    'Price': priceList,
    'Location': locationList,
    'Condition': conditionList,
    'Colors': colorsList,
    'Sizes': sizesList,
    'Image': imageList,
    'Product Link': linkList
}

dataframe = pd.DataFrame(data)

In [ ]:
dataframe.to_csv('Data Products.csv', index=False)